In [30]:
from math import log
def createDataSet():
    data = {
        '色泽': ['0', '1', '1', '0', '2', '0', '1', '1', '1', '0', '2', '2', '0', '2', '1', '2', '0'],
        '根蒂': ['0', '0', '0', '0', '0', '1', '1', '1', '1', '2', '2', '0', '1', '1', '1', '0', '0'],
        '敲声': ['0', '1', '0', '1', '0', '0', '0', '0', '1', '2', '2', '0', '0', '1', '0', '0', '1'],
        '纹理': ['0', '0', '0', '0', '0', '0', '1', '0', '1', '0', '2', '2', '1', '1', '0', '2', '1'],
        '脐部': ['0', '0', '0', '0', '0', '1', '1', '1', '1', '2', '2', '2', '0', '0', '1', '2', '1'],
        '触感': ['0', '0', '0', '0', '0', '1', '1', '0', '0', '1', '0', '1', '0', '0', '1', '0', '0'],
        '好瓜': [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    }

    la = ['色泽', '根蒂', '敲声', '纹理', '脐部', '触感']
    
    dataSet = []
    for i in range(len(data['好瓜'])):
        temp = []
        for j in la:
            temp.append(data[j][i])
        temp.append(data['好瓜'][i])
        dataSet.append(temp)
    return dataSet, la


"""
函数说明:计算给定数据集的经验熵(香农熵)
Parameters:
    dataSet - 数据集
Returns:
    shannonEnt - 经验熵(香农熵)
"""
def calcShannonEnt(dataSet):
    numEntires = len(dataSet)                        #返回数据集的行数
    labelCounts = {}                                 #保存每个标签(Label)出现次数的字典
    for featVec in dataSet:                          #对每组特征向量进行统计
        currentLabel = featVec[-1]                   #提取标签(Label)信息
        if currentLabel not in labelCounts.keys():   #如果标签(Label)没有放入统计次数的字典,添加进去
            labelCounts[currentLabel] = 0
        labelCounts[currentLabel] += 1               #Label计数
    shannonEnt = 0.0                                 #经验熵(香农熵)
    for key in labelCounts:                          #计算香农熵
        prob = float(labelCounts[key]) / numEntires  #选择该标签(Label)的概率
        shannonEnt -= prob * log(prob, 2)            #利用公式计算
    return shannonEnt                                #返回经验熵(香农熵)
 
 
"""
函数说明:按照给定特征划分数据集
Parameters:
    dataSet - 待划分的数据集
    axis - 划分数据集的特征
    value - 需要返回的特征的值
"""
def splitDataSet(dataSet, axis, value):
    retDataSet = []                                     #创建返回的数据集列表
    for featVec in dataSet:                             #遍历数据集
        if featVec[axis] == value:
            reducedFeatVec = featVec[:axis]             #去掉axis特征
            reducedFeatVec.extend(featVec[axis+1:])     #将符合条件的添加到返回的数据集
            retDataSet.append(reducedFeatVec)
    return retDataSet                                   #返回划分后的数据集
 
 
"""
函数说明:选择最优特征
Parameters:
    dataSet - 数据集
Returns:
    bestFeature - 信息增益最大的(最优)特征的索引值
"""
def chooseBestFeatureToSplit(dataSet):
    numFeatures = len(dataSet[0]) - 1                     #特征数量
    baseEntropy = calcShannonEnt(dataSet)                 #计算数据集的香农熵
    bestInfoGain = 0.0                                    #信息增益
    bestFeature = -1                                      #最优特征的索引值
    for i in range(numFeatures):                          #遍历所有特征
        #获取dataSet的第i个所有特征
        featList = [example[i] for example in dataSet]
        uniqueVals = set(featList)                         #创建set集合{},元素不可重复
        newEntropy = 0.0                                   #经验条件熵
        for value in uniqueVals:                           #计算信息增益
            subDataSet = splitDataSet(dataSet, i, value)           #subDataSet划分后的子集
            prob = len(subDataSet) / float(len(dataSet))           #计算子集的概率
            newEntropy += prob * calcShannonEnt(subDataSet)        #根据公式计算经验条件熵
        infoGain = baseEntropy - newEntropy                        #信息增益
        print("第%d个特征的增益为%.3f" % (i, infoGain))             #打印每个特征的信息增益
        if (infoGain > bestInfoGain):                              #计算信息增益
            bestInfoGain = infoGain                                #更新信息增益，找到最大的信息增益
            bestFeature = i                                        #记录信息增益最大的特征的索引值
    return bestFeature                                             #返回信息增益最大的特征的索引值
 
 
if __name__ == '__main__':
    dataSet, features = createDataSet()
    entropy=calcShannonEnt(dataSet)
    bestfeature=chooseBestFeatureToSplit(dataSet)
    print("训练集的熵为:%f"%(entropy))
    print("最优特征索引值:" + str(bestfeature))

第0个特征的增益为0.108
第1个特征的增益为0.143
第2个特征的增益为0.141
第3个特征的增益为0.381
第4个特征的增益为0.289
第5个特征的增益为0.006
训练集的熵为:0.997503
最优特征索引值:3


In [42]:
import pydotplus
from black.trans import StringID
from pandas import DataFrame
from sklearn import tree
from sklearn.feature_extraction import DictVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import export_graphviz
import pandas as pd
import numpy as np

# 打开并读取数据文件
with open('xigua.txt',encoding='utf-8')  as fr:
    lenses = [inst.strip().split('\t') for inst in fr.readlines()]
lenses
# 提取目标变量
lenses_target = []
for each in lenses:
    lenses_target.append(each[-1])
from math import log2

# 定义特征标签
lensesLabels = ['色泽', '根蒂', '敲声', '纹理', '脐部', '触感']

# 将数据转换为字典格式，以便于创建DataFrame
lenses_list=[]
lenses_dict = {}
for each_label in lensesLabels:
    for each in lenses:
        # 然后，对lensesLabels中的每个标签进行迭代。对于每个标签，再对lenses中的每个元素进行迭代，将与当前标签对应的元素添加到lenses_list中。
        lenses_list.append(each[lensesLabels.index(each_label)])
    # 接下来，将lenses_list作为值，当前的标签作为键，添加到lenses_dict中。
    lenses_dict[each_label] = lenses_list
    lenses_list=[]
# 创建DataFrame
lenses_pd = pd.DataFrame(lenses_dict)

# 使用LabelEncoder将字符串标签转换为数字
le = LabelEncoder()
for col in lenses_pd.columns:
    lenses_pd[col] = le.fit_transform(lenses_pd[col])

# 创建并训练决策树分类器
# clf = tree.DecisionTreeClassifier()
# clf = clf.fit(lenses_pd.values.tolist(), lenses_target)


class Node:
    def __init__(self, predicted_class):
        self.predicted_class = predicted_class
        self.feature_index = 0
        self.threshold = 0
        self.left = None
        self.right = None


class DecisionTreeClassifier:
    def __init__(self, max_depth=None):
        self.max_depth = max_depth
    # fit方法用于训练决策树
    def fit(self, X, y):
        # 计算类别的数量
        self.n_classes_ = len(set(y))
        # 计算特征的数量
        self.n_features_ = X.shape[1]
        # 创建决策树
        self.tree_ = self._grow_tree(X, y)

    # _grow_tree方法用于创建决策树
    def _grow_tree(self, X, y, depth=0):
        # 计算每个类别的样本数量
        num_samples_per_class = [np.sum(y == i) for i in range(self.n_classes_)]
        # 预测的类别是样本最多的类别
        predicted_class = np.argmax(num_samples_per_class)
        # 创建一个新的节点
        node = Node(predicted_class=predicted_class)
        # 如果没有达到最大深度，继续创建子节点
        if depth < self.max_depth:
            # 找到最佳的分割点
            idx, thr = self._best_split(X, y)
            if idx is not None:
                # 根据最佳的分割点分割数据
                indices_left = X[:, idx] < thr
                X_left, y_left = X[indices_left], y[indices_left]
                X_right, y_right = X[~indices_left], y[~indices_left]
                # 创建左子节点和右子节点
                node.feature_index = idx
                node.threshold = thr
                node.left = self._grow_tree(X_left, y_left, depth + 1)
                node.right = self._grow_tree(X_right, y_right, depth + 1)
        # 返回创建的节点
        return node
    # _best_split方法用于找到最佳的分割点
    def _best_split(self, X, y):
        m = y.size  # 获取样本的数量
        if m <= 1:  # 如果样本数量小于等于1，那么就没有必要进行分割，直接返回None, None
            return None, None

        num_parent = [np.sum(y == c) for c in range(self.n_classes_)]  # 计算每个类别的样本数量
        best_gain = 0.0  # 初始化最大的信息增益为0
        best_idx, best_thr = None, None  # 初始化最佳的特征和阈值为None

        # 遍历所有的特征
        for idx in range(self.n_features_):
            # 对特征的所有可能值进行排序，然后计算每个可能的分割点的信息增益
            thresholds, classes = zip(*sorted(zip(X[:, idx], y)))
            # print(X, y)
            # print('2222222222222222222222')
            # print(thresholds, classes)
            print('------------------------')
            print( classes)
            num_left = [0] * self.n_classes_  # 初始化左子节点的每个类别的样本数量为0
            num_right = num_parent.copy()  # 右子节点的每个类别的样本数量初始为父节点的样本数量
            for i in range(1, m):  # 遍历每个可能的分割点
                c = classes[i - 1]
                num_left[c] += 1  # 更新左子节点的样本数量
                num_right[c] -= 1  # 更新右子节点的样本数量
                # 计算左子节点和右子节点的熵
                ent_left = -sum(
                    (num_left[x] / i) * log2(num_left[x] / i)
                    for x in range(self.n_classes_)
                    if num_left[x] > 0)
                ent_right = -sum(
                    (num_right[x] / (m - i)) * log2(num_right[x] / (m - i))
                    for x in range(self.n_classes_)
                    if num_right[x] > 0)
                # 计算分割后的熵
                ent = (i * ent_left + (m - i) * ent_right) / m
                # 计算信息增益
                ig = -sum(
                    (num_parent[x] / m) * log2(num_parent[x] / m)
                    for x in range(self.n_classes_)
                    if num_parent[x] > 0) - ent
                if thresholds[i] == thresholds[i - 1]:  # 如果两个连续的阈值相同，那么就跳过这个阈值
                    continue
                if ig > best_gain:  # 如果当前的信息增益大于最大的信息增益，那么就更新最大的信息增益、特征和阈值
                    best_gain = ig
                    best_idx = idx
                    best_thr = (thresholds[i] + thresholds[i - 1]) / 2
        # 返回最佳的特征和阈值
        return best_idx, best_thr

# 使用LabelEncoder将字符串标签转换为数字
le = LabelEncoder()
for col in lenses_pd.columns:
    lenses_pd[col] = le.fit_transform(lenses_pd[col])

# 将目标变量也转换为数字
lenses_target = le.fit_transform(lenses_target)


clf = DecisionTreeClassifier(max_depth=3)
clf.fit(lenses_pd.values, lenses_target)



# # 导出决策树的图形表示
# dot_data = tree.export_graphviz(clf, out_file=None, filled=True, rounded=True,
#                                 feature_names=lenses_pd.keys(), class_names=clf.classes_,
#                                 fontname="SimHei")

# # 使用pydotplus创建图形并保存为PDF文件
# graph = pydotplus.graph_from_dot_data(dot_data)
# graph.write_pdf('wangjing.pdf')

------------------------
(0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1)
------------------------
(0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1)
------------------------
(0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0)
------------------------
(0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1)
------------------------
(0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1)
------------------------
(0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1)
------------------------
(0, 0, 0)
------------------------
(0, 0, 0)
------------------------
(0, 0, 0)
------------------------
(0, 0, 0)
------------------------
(0, 0, 0)
------------------------
(0, 0, 0)
------------------------
(0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1)
------------------------
(0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1)
------------------------
(0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0)
------------------------
(0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1)
------------------------
(0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 